# LSTM을 활용한 교통 변화 예측

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 

In [ ]:
traffic_data_2015_01=pd.read_csv('data/2015_01분기.csv',encoding='cp949')
traffic_data_2015_02=pd.read_csv('data/2015_02분기.csv',encoding='cp949')
traffic_data_2015_03=pd.read_csv('data/2015_03분기.csv',encoding='cp949')
traffic_data_2015_04=pd.read_csv('data/2015_04분기.csv',encoding='cp949')
traffic_data_2016_01=pd.read_csv('data/2016_01분기.csv',encoding='cp949')
traffic_data_2016_02=pd.read_csv('data/2016_02분기.csv',encoding='cp949')
traffic_data_2016_03=pd.read_csv('data/2016_03분기.csv',encoding='cp949')
traffic_data_2016_04=pd.read_csv('data/2016_04분기.csv',encoding='cp949')
traffic_data_2017_01=pd.read_csv('data/2017_01분기.csv',encoding='cp949')
traffic_data_2017_02=pd.read_csv('data/2017_02분기.csv',encoding='cp949')
traffic_data_2017_03=pd.read_csv('data/2017_03분기.csv',encoding='cp949')
traffic_data_2017_04=pd.read_csv('data/2017_04분기.csv',encoding='cp949')
traffic_data_2018_01=pd.read_csv('data/2018_01분기.csv',encoding='cp949')
traffic_data_2018_02=pd.read_csv('data/2018_02분기.csv',encoding='cp949')
traffic_data_2018_03=pd.read_csv('data/2018_03분기.csv',encoding='cp949')
traffic_data_2018_04=pd.read_csv('data/2018_04분기.csv',encoding='cp949')
traffic_data_2019_01=pd.read_csv('data/2019_01분기.csv',encoding='cp949')
traffic_data_2019_02=pd.read_csv('data/2019_02분기.csv',encoding='cp949')
traffic_data_2019_03=pd.read_csv('data/2019_03분기.csv',encoding='cp949')
traffic_data_2019_04=pd.read_csv('data/2019_04분기.csv',encoding='cp949')
traffic_data_2020_01=pd.read_csv('data/2020_01분기.csv',encoding='cp949')
traffic_data_2020_02=pd.read_csv('data/2020_02분기.csv',encoding='cp949')
traffic_data_2020_03=pd.read_csv('data/2020_03분기.csv',encoding='cp949')
traffic_data_2020_04=pd.read_csv('data/2020_04분기.csv',encoding='cp949')
traffic_data_2021_01=pd.read_csv('data/2021_01분기.csv',encoding='cp949')
traffic_data_2021_02=pd.read_csv('data/2021_02분기.csv',encoding='cp949')

In [ ]:
traffic_data_2015=pd.concat([traffic_data_2015_01,traffic_data_2015_02,traffic_data_2015_03,traffic_data_2015_04])
traffic_data_2016=pd.concat([traffic_data_2016_01,traffic_data_2016_02,traffic_data_2016_03,traffic_data_2016_04])
traffic_data_2017=pd.concat([traffic_data_2017_01,traffic_data_2017_02,traffic_data_2017_03,traffic_data_2017_04])
traffic_data_2018=pd.concat([traffic_data_2018_01,traffic_data_2018_02,traffic_data_2018_03,traffic_data_2018_04])
traffic_data_2019=pd.concat([traffic_data_2019_01,traffic_data_2019_02,traffic_data_2019_03,traffic_data_2019_04])
traffic_data_2020=pd.concat([traffic_data_2020_01,traffic_data_2020_02,traffic_data_2020_03,traffic_data_2020_04])
traffic_data_2021=pd.concat([traffic_data_2021_01,traffic_data_2021_02])
traffic_data_past=pd.concat([traffic_data_2015,traffic_data_2016,traffic_data_2017,traffic_data_2018,traffic_data_2019])

In [ ]:
traffic_data_past.head()

In [ ]:
traffic_data_past['집계일자']=pd.to_datetime(traffic_data_past['집계일자'],format='%Y%m%d')
traffic_data_past['연도']=traffic_data_past['집계일자'].dt.year

In [ ]:
plt.figure(figsize=(16, 9))
sns.lineplot(y=traffic_data_past['1종교통량'], x=traffic_data_past['집계일자'])
plt.xlabel('time')
plt.ylabel('type 1 traffic')

In [ ]:
traffic_data_past=traffic_data_past.replace('서서울영업소','서서울')
traffic_data_past=traffic_data_past.replace('서울영업소','서울')
traffic_data_past=traffic_data_past.replace('서대구영업소','서대구')
traffic_data_past=traffic_data_past.replace('북대구영업소','북대구')
traffic_data_past=traffic_data_past.replace('남대구영업소','남대구')

In [ ]:
seoul_traffic_data_past=traffic_data_past[(traffic_data_past['영업소명']=='서서울')|(traffic_data_past['영업소명']=='서울')|(traffic_data_past['영업소명']=='남서울')
]

In [ ]:
from sklearn.preprocessing import MinMaxScaler #정규화 과정

seoul_traffic_data_past.sort_index(ascending=False).reset_index(drop=True)

scaler = MinMaxScaler()
scale_cols = ['1종교통량','2종교통량','3종교통량','총교통량']
df_scaled = scaler.fit_transform(seoul_traffic_data_past[scale_cols])
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

df_scaled

In [ ]:
TEST_SIZE = 200
WINDOW_SIZE = 20

train = df_scaled[:-TEST_SIZE]
test = df_scaled[-TEST_SIZE:]

In [ ]:
def make_dataset(data, label, window_size=20):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [ ]:
from sklearn.model_selection import train_test_split

feature_cols = ['1종교통량','2종교통량','3종교통량']
label_cols = ['총교통량']

train_feature = train[feature_cols]
train_label = train[label_cols]

train_feature, train_label = make_dataset(train_feature, train_label, 20)

x_train, x_valid, y_train, y_valid = train_test_split(train_feature, train_label, test_size=0.2)
x_train.shape, x_valid.shape

In [ ]:
test_feature = test[feature_cols]
test_label = test[label_cols]

test_feature.shape, test_label.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(16, 
               input_shape=(train_feature.shape[1], train_feature.shape[2]), 
               activation='relu', 
               return_sequences=False)
          )

model.add(Dense(1))

In [ ]:
import os

model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='val_loss', patience=5)

model_path = 'model'
filename = os.path.join(model_path, 'tmp_checkpoint.h5')
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = model.fit(x_train, y_train,
                                    epochs=100,
                                    batch_size=16,
                                    validation_data=(x_valid, y_valid), 
                                    callbacks=[early_stop, checkpoint])